In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393168 sha256=9a0d78d2077f2934eee51dd4c89349b777dfbf47aa61737eee883828284da12b
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import csv
import fasttext
import os
import string
import nltk

In [ ]:
os.environ["KAGGLE_USERNAME"]="nidamanuriadarsh"
os.environ["KAGGLE_KEY"]="afc73a9685f30971965a4c23de238034"

In [ ]:
!kaggle datasets download -d vaibhavkumar11/hindi-english-parallel-corpus

 94% 105M/112M [00:00<00:00, 149MB/s] 
100% 112M/112M [00:00<00:00, 138MB/s]


In [ ]:
!unzip hindi-english-parallel-corpus.zip

Archive:  hindi-english-parallel-corpus.zip
  inflating: hindi_english_parallel.csv  


In [ ]:
df = pd.read_csv("hindi_english_parallel.csv")

In [ ]:
class Processor:
  def __init__(self,data):
    self.data = data
    self.corpus=[]
    self.fintext=[]
    self.processed_data = []
  def process(self):
    self.remove_unwanted()
    self.generate_fintext()
    return self.fintext
  def remove_unwanted(self):
    self.processed_data= [ re.sub("\w*\d\w*","",re.sub("@[\w\-]+","User",re.sub("\n","",re.sub('[%s]' % re.escape(string.punctuation), '', re.sub("'\[.*?\]'","",re.sub('https?://\S+|www\.\S+',"",re.sub('<.*?>+', '', str(i)))))))).lower().replace("#","")  for i in self.data]

  def generate_fintext(self):
    pattern = re.compile(r"(.)\1{2,}")
    reduced_words =[]
    fintext = []
    for i in self.processed_data:
      reduced_words = [ pattern.sub(r"\1\1",j) for j in i.split()]
      for k in reduced_words:
        self.corpus.append(k)
      fintext.append(' '.join(reduced_words))
    self.fintext = fintext


In [ ]:
df["refined"]=Processor(df["english"].to_list()).process()

In [ ]:
def save_sentences_to_file(sentences, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for sentence in sentences:
            file.write(str(sentence) + '\n')

In [ ]:
save_sentences_to_file(df["english"].to_list()[:100000],"english_sentences.txt")

In [ ]:
df.head()

,hindi,english,refined
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel,the default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default,a list of plugins that are disabled by default


In [ ]:
def train_embeddings(language, input_file, output_file):
    model = fasttext.train_unsupervised(input=input_file, model='skipgram',dim=300)
    model.save_model(output_file)

# train_embeddings('english', 'english_sentences.txt', 'english_embeddings.bin')

In [ ]:
# model = fasttext.load_model('english_embeddings.bin')

In [ ]:
# english_vectors = {}

# for word in model.words:
#     vector = model.get_word_vector(word)
#     english_vectors[word] = vector

In [ ]:
!unzip Hinglish-20230621T161556Z-001.zip

Archive:  Hinglish-20230621T161556Z-001.zip
  inflating: Hinglish/agr_hi_fb_gold_E.csv  
  inflating: Hinglish/trac_2022_annotator_8_dev_E.csv  
  inflating: Hinglish/agr_hi_dev_E.csv  
  inflating: Hinglish/trac_2022_annotator_9_dev_E.csv  
  inflating: Hinglish/trac_2022_annotator_7_dev_E.csv  
  inflating: Hinglish/trac_2022_annotator_9_train_E.csv  
  inflating: Hinglish/test_tweets_3.csv  
  inflating: Hinglish/agr_hi_tw_gold_E.csv  
  inflating: Hinglish/trac_2022_annotator_8_train_E.csv  
  inflating: Hinglish/agr_hi_train_E.csv  
  inflating: Hinglish/trac_2022_annotator_7_train_E.csv  
  inflating: Hinglish/hate_speech.csv  


In [ ]:
import os

folder_path = '/content/Hinglish/'

file_names = os.listdir(folder_path)

for file_name in file_names:
    print(file_name)

trac_2022_annotator_7_train_E.csv
agr_hi_fb_gold_E.csv
hate_speech.csv
trac_2022_annotator_8_train_E.csv
trac_2022_annotator_9_train_E.csv
agr_hi_tw_gold_E.csv
trac_2022_annotator_8_dev_E.csv
trac_2022_annotator_7_dev_E.csv
agr_hi_train_E.csv
trac_2022_annotator_9_dev_E.csv
test_tweets_3.csv
agr_hi_dev_E.csv


In [ ]:

L = []
for i in ["trac_2022_annotator_7_train_E.csv", "trac_2022_annotator_8_train_E.csv", "trac_2022_annotator_9_train_E.csv"]:
    path = "/content/Hinglish/" + i
    df1 = pd.read_csv(path)
    L.append(df1)

combined_df = pd.concat(L)
combined_df.head()

,Unnamed: 0,id,text,Aggression,Aggression Intensity,Discursive Role,Gender Bias,Communal Bias,Caste/Class Bias,Ethnicity/ Racial Bias
0,0,facebook_corpus_msr_747593,Shaikh Bhai.. Subha Kab Ki Ho gyi Hai.... Man...,OAG,NtAG,ATK,NGEN,NCOM,NCAS,NETH
1,1,facebook_corpus_msr_2111297,No India is better off in BJPs hands but it is...,OAG,NtAG,ATK,NGEN,NCOM,NCAS,NETH
2,2,facebook_corpus_msr_1450799,madam ji Sonia ganghi jeetegi tabhi to banegi ...,OAG,NtAG,ATK,NGEN,NCOM,NCAS,NETH
3,3,facebook_corpus_msr_956362,Now i really belive 5vi pass pm ki degree nak...,CAG,NtAG,NaN,NGEN,NCOM,NCAS,NETH
4,4,facebook_corpus_msr_1972724,Reasons: 1.rahul 2. Rahul 3.rahul 4.rahil . . ...,CAG,NtAG,NaN,NGEN,NCOM,NCAS,NETH


In [ ]:
print(len(combined_df["text"].to_list()))

32813


In [ ]:
combined_df["refined"]=Processor(combined_df["text"].to_list()).process()

In [ ]:
combined_df.head()

,Unnamed: 0,id,text,Aggression,Aggression Intensity,Discursive Role,Gender Bias,Communal Bias,Caste/Class Bias,Ethnicity/ Racial Bias,refined
0,0,facebook_corpus_msr_747593,Shaikh Bhai.. Subha Kab Ki Ho gyi Hai.... Man...,OAG,NtAG,ATK,NGEN,NCOM,NCAS,NETH,shaikh bhai subha kab ki ho gyi hai manipur me...
1,1,facebook_corpus_msr_2111297,No India is better off in BJPs hands but it is...,OAG,NtAG,ATK,NGEN,NCOM,NCAS,NETH,no india is better off in bjps hands but it is...
2,2,facebook_corpus_msr_1450799,madam ji Sonia ganghi jeetegi tabhi to banegi ...,OAG,NtAG,ATK,NGEN,NCOM,NCAS,NETH,madam ji sonia ganghi jeetegi tabhi to banegi ...
3,3,facebook_corpus_msr_956362,Now i really belive 5vi pass pm ki degree nak...,CAG,NtAG,NaN,NGEN,NCOM,NCAS,NETH,now i really belive pass pm ki degree nakli hai
4,4,facebook_corpus_msr_1972724,Reasons: 1.rahul 2. Rahul 3.rahul 4.rahil . . ...,CAG,NtAG,NaN,NGEN,NCOM,NCAS,NETH,reasons rahul upto infinite


In [ ]:
save_sentences_to_file(combined_df["refined"].to_list(),"hinglish_sentences.txt")

In [ ]:
# import numpy as np

# english_vectors_file = "english_vectors.vec"
# with open(english_vectors_file, "w", encoding="utf-8") as f:
#     for word in model.words:
#         vector = model.get_word_vector(word)
#         vector_str = " ".join(str(x) for x in vector)
#         line = f"{word} {vector_str}\n"
#         f.write(line)

In [ ]:
hinglish_corpus = "/content/hinglish_sentences.txt"
eng_file = "/content/english_embeddings.bin"

# english_model = fasttext.load_model("english_embeddings.bin")

# Create a new FastText model
hinglish_model = fasttext.train_unsupervised(
    hinglish_corpus,
    dim=300,  # Adjust the dimensionality as needed
    epoch=10
)





In [ ]:
hinglish_model.get_nearest_neighbors('namaste')
# hinglish_model.get_analogies("India","USA","Russia")

[(0.8782214522361755, 'namste'),
 (0.6415764689445496, '🤨🤨'),
 (0.6411495804786682, 'namak'),
 (0.6408529281616211, 'taste'),
 (0.6241011023521423, 'trumph'),
 (0.6177101135253906, 'raste'),
 (0.6141558289527893, 'nama'),
 (0.603705108165741, 'jite'),
 (0.5952069163322449, 'namaj'),
 (0.5917879343032837, 'namo')]

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 72% 58.0M/80.9M [00:00<00:00, 187MB/s]
100% 80.9M/80.9M [00:00<00:00, 208MB/s]


In [ ]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
df3 = pd.read_csv("training.1600000.processed.noemoticon.csv",encoding='latin-1')

In [ ]:
df3.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [ ]:
Lf = Processor(df3["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"].to_list()).process()

In [ ]:
def save_sentences_to_file(sentences, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for sentence in sentences:
            file.write(str(sentence) + '\n')

In [ ]:
save_sentences_to_file(Lf[:100000],"english_sentences1.txt")

In [ ]:
def train_embeddings(language, input_file, output_file):
    model = fasttext.train_unsupervised(input=input_file, model='skipgram',dim=300)
    model.save_model(output_file)

In [ ]:
train_embeddings('english', 'english_sentences1.txt', 'english_embeddings1.bin')

In [ ]:
model_test =fasttext.load_model('english_embeddings1.bin')

In [ ]:
model_test.get_nearest_neighbors("hello")

[(0.9377654194831848, 'helloo'),
 (0.814643919467926, 'hellivina'),
 (0.8059364557266235, 'hella'),
 (0.8031098246574402, 'welcome'),
 (0.7978597283363342, 'welcomed'),
 (0.7911489605903625, 'iamdiddy'),
 (0.7900818586349487, 'jello'),
 (0.7780429720878601, 'hollywoodtrey'),
 (0.7779256105422974, 'lovebscott'),
 (0.7759992480278015, 'emm')]

In [ ]:
model_test.get_analogies("house","apartment","building")

[(0.7341111302375793, 'household'),
 (0.6840641498565674, 'warehouse'),
 (0.6803001165390015, 'bu'),
 (0.6749755144119263, 'cleaned'),
 (0.667015016078949, 'houses'),
 (0.6649080514907837, 'hundred'),
 (0.6641660928726196, 'cabinet'),
 (0.6613180041313171, 'syracuse'),
 (0.6590073704719543, 'budget'),
 (0.6574659943580627, 'squad')]

In [ ]:
hinglish_model.save_model("hinglish_embeddings1.bin")

In [ ]:
pretrained_dimension = model_test.get_dimension()
print("Pretrained Embeddings Dimension:", pretrained_dimension)

Pretrained Embeddings Dimension: 300


In [ ]:
import fasttext

pretrained_model_path = 'english_embeddings1.bin'
output_file_path = 'english_vectors.txt'

model = fasttext.load_model(pretrained_model_path)
words = model.get_words()

with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for word in words:
        vector = model.get_word_vector(word)
        vector_str = ' '.join(str(val) for val in vector)
        output_file.write(f"{word} {vector_str}\n")


In [ ]:
# import fasttext
# hinglish_corpus = "/content/hinglish_sentences.txt"
# # Step 1: Convert pre-trained FastText binary model to text format
# pretrained_model_path = 'english_embeddings1.bin'
# text_format_model_path = 'english_vectors_model.bin'

# model = fasttext.load_model(pretrained_model_path)
# model.save_model(text_format_model_path, ft_format="vectors")

# # Step 2: Train the Hinglish model using the text format model
# # hinglish_corpus = 'hinglish_corpus.txt'
# hinglish_model = fasttext.train_unsupervised(
#     hinglish_corpus,
#     pretrained_vectors=text_format_model_path,
#     dim=300,  # Adjust the dimensionality as needed
#     epoch=10
# )

# # Step 3: Save the Hinglish model
# # hinglish_model.save_model('hinglish_model.bin')

TypeError: ignored

In [ ]:
import fasttext

# Load the FastText binary model
fasttext_model_path = 'english_embeddings1.bin'
model = fasttext.load_model(fasttext_model_path)

# Save the FastText embeddings in .vec format
vec_file_path = 'en.vec'
model.save_model(vec_file_path)


In [ ]:
import fasttext

# Load the FastText binary model
fasttext_model_path = 'hinglish_embeddings1.bin'
model = fasttext.load_model(fasttext_model_path)

# Save the FastText embeddings in .vec format
vec_file_path = 'hi.vec'
model.save_model(vec_file_path)


In [ ]:
import fasttext

# Load the FastText model in .vec format
vec_file_path = 'en.vec'
model = fasttext.load_model(vec_file_path)

# Save the FastText embeddings in the traditional Word2Vec text format
word2vec_text_format_path = 'en.txt'
model.save_model(word2vec_text_format_path)


TypeError: ignored

In [ ]:
word2vec_text_format_path = 'en.txt'
model.save_model(word2vec_text_format_path)

In [ ]:
vec_file_path = 'hi.vec'
model = fasttext.load_model(vec_file_path)

# Save the FastText embeddings in the traditional Word2Vec text format
word2vec_text_format_path = 'hi.txt'
model.save_model(word2vec_text_format_path)

In [ ]:
from gensim.models import FastText
from gensim.utils import simple_preprocess
from gensim.test.utils import get_tmpfile


In [ ]:
model = FastText(vector_size=300, window=5, min_count=5, workers=4)


In [ ]:
def read_corpus(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            yield simple_preprocess(line)


In [ ]:
model.build_vocab(corpus_file='english_sentences1.txt')
total_words = model.corpus_total_words
model.train(corpus_file='english_sentences1.txt', total_words=total_words, epochs=10)


(9292577, 13199070)

In [ ]:
model.save("en1.vec")
model.wv.save_word2vec_format('en2.bin')

In [ ]:
model.build_vocab(corpus_file='hinglish_sentences.txt',update=True)

In [ ]:
new_ten = model.corpus_total_words
print((total_words,new_ten))

(1311134, 469413)


In [ ]:
model.train(corpus_file='hinglish_sentences.txt', total_words=int(1311134-469413), epochs=10)

(5899932, 7658500)

In [ ]:
model.wv.most_similar("kutta")

[('kuta', 0.8874878287315369),
 ('dutta', 0.828890860080719),
 ('kutto', 0.797985851764679),
 ('ptta', 0.7546012997627258),
 ('kutaa', 0.7326545119285583),
 ('kurta', 0.7244449257850647),
 ('kutti', 0.7206964492797852),
 ('kuto', 0.7187777161598206),
 ('gta', 0.715463399887085),
 ('bnta', 0.7123579978942871)]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
print(model.wv["cat"])

[-1.22413659e+00  7.87053781e-04  2.53562033e-01  4.06382382e-01
  4.42450136e-01 -3.48980695e-01 -2.11276874e-01 -2.33722374e-01
  2.90653378e-01  4.80590820e-01  5.03670983e-02 -1.34551167e+00
  6.90992475e-01  3.58081341e-01 -3.44301850e-01  2.81448364e-01
  5.29660642e-01 -3.21456715e-02  8.56540024e-01 -8.02800715e-01
  1.76306915e+00 -5.68041325e-01  5.48489392e-01  1.71619445e-01
 -2.50730729e+00 -1.04635799e+00  5.57624936e-01 -1.19473147e+00
 -1.49312317e+00  4.29821730e-01  1.22461414e+00  4.97484058e-01
  4.47150439e-01 -1.19516861e+00 -2.17410088e-01 -1.67570993e-01
  5.00277638e-01 -4.23618942e-01 -9.27585423e-01 -6.68963552e-01
 -3.12783793e-02  4.41698402e-01  3.40262264e-01  8.28623399e-02
 -1.65462089e+00  2.01546836e+00  1.00473844e-01  8.43908114e-04
 -3.26051563e-01 -4.50016588e-01  9.08124030e-01  4.56947744e-01
 -1.45320863e-01 -3.84270906e-01  2.09032089e-01 -1.59522378e+00
  4.14931238e-01 -7.14612663e-01 -4.15295422e-01  6.76241696e-01
 -6.94460213e-01  1.55971

In [ ]:
print(cosine_similarity([model.wv["cat"]],[model.wv["billi"]]))
print(cosine_similarity([model.wv["cat"]],[model.wv["kutta"]]))
print(cosine_similarity([model.wv["namaste"]],[model.wv["dog"]]))

[[0.19662327]]
[[-0.14458239]]
[[0.15767942]]


In [ ]:
model.wv.most_similar("kutta")

[('kuta', 0.8874878287315369),
 ('dutta', 0.828890860080719),
 ('kutto', 0.797985851764679),
 ('ptta', 0.7546012997627258),
 ('kutaa', 0.7326545119285583),
 ('kurta', 0.7244449257850647),
 ('kutti', 0.7206964492797852),
 ('kuto', 0.7187777161598206),
 ('gta', 0.715463399887085),
 ('bnta', 0.7123579978942871)]

In [ ]:
model1 = FastText(vector_size=300, window=5, min_count=5, workers=4)


In [ ]:
model1.build_vocab(corpus_file='hinglish_sentences.txt')
total_words = model1.corpus_total_words
model1.train(corpus_file='hinglish_sentences.txt', total_words=total_words, epochs=10)


(3665606, 4818660)

In [ ]:
model1.save("hi1.vec")
model1.wv.save_word2vec_format('hi2.bin')

In [ ]:
model.save("en-hi1.vec")
model.wv.save_word2vec_format('hien2.bin')

In [ ]:
path="en1.vec"
with open(path,encoding='latin-1') as f:
        first_line = f.readline()
        first_line = first_line.split(' ')
        print(first_line)

['\x80\x04\x95\x01\x00\x01\x00\x00\x00\x00\x00\x8c\x16gensim.models.fasttext\x94\x8c\x08FastText\x94\x93\x94)\x81\x94}\x94(\x8c\x04load\x94\x8c\x0cgensim.utils\x94\x8c\x12call_on_class_only\x94\x93\x94\x8c\x14load_fasttext_format\x94h\x08\x8c\tcallbacks\x94)\x8c\x0bword_ngrams\x94K\x01\x8c\x02wv\x94h\x00\x8c\x14FastTextKeyedVectors\x94\x93\x94)\x81\x94}\x94(\x8c\x0bvector_size\x94M,\x01\x8c\x0cindex_to_key\x94]\x94(\x8c\x01i\x94\x8c\x02to\x94\x8c\x03the\x94\x8c\x01a\x94\x8c\x02my\x94\x8c\x03and\x94\x8c\x02is\x94\x8c\x02in\x94\x8c\x02it\x94\x8c\x03for\x94\x8c\x02im\x94\x8c\x02of\x94\x8c\x03you\x94\x8c\x02me\x94\x8c\x02on\x94\x8c\x04have\x94\x8c\x02so\x94\x8c\x03but\x94\x8c\x03not\x94\x8c\x04that\x94\x8c\x02at\x94\x8c\x04just\x94\x8c\x03was\x94\x8c\x02be\x94\x8c\x03its\x94\x8c\x04this\x94\x8c\x02up\x94\x8c\x04with\x94\x8c\x03now\x94\x8c\x04work\x94\x8c\x02no\x94\x8c\x02go\x94\x8c\x03day\x94\x8c\x03get\x94\x8c\x03out\x94\x8c\x04cant\x94\x8c\x04dont\x94\x8c\x03all\x94\x8c\x04like\x94\x8c\x